In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [2]:
# 虚构数据集生成器
def generate_synthetic_data(num_samples=2666, num_features=26):
    np.random.seed(42)
    X = np.random.randn(num_samples, num_features) * 2 + 1
    # 创建非线性目标函数
    y = (X[:, 0] ** 2 + np.sin(X[:, 1] * 2) + 
         X[:, 5] * X[:, 7] + 0.5 * np.random.randn(num_samples))
    return X, y

In [3]:
# 数据加载器
class FeatureDataset(Dataset):
    def __init__(self, features, targets):
        self.X = torch.FloatTensor(features)
        self.y = torch.FloatTensor(targets)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [4]:
# 位置编码模块
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=26):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [5]:
# Transformer回归模型
class TransformerRegressor(nn.Module):
    def __init__(self, input_dim=26, d_model=128, nhead=4, num_layers=3):
        super().__init__()
        self.feature_embedding = nn.Linear(1, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model, nhead, dim_feedforward=4*d_model, dropout=0.1, batch_first=True)
        
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.regressor = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = self.feature_embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.regressor(x).squeeze(-1)

In [6]:
def train(model, loader, criterion, optimizer, scheduler, epochs):
    model.train()
    losses = []
    for epoch in range(epochs):
        epoch_loss = 0
        for batch_X, batch_y in loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        losses.append(epoch_loss / len(loader))
        scheduler.step()
        print(f'Epoch {epoch+1} | Loss: {loss.item():.2f}')
    return losses

In [7]:
# 训练损失可视化
def plot_loss_curve(train_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.title('Training Loss Curve')
    plt.legend()
    plt.grid(True)
    
    plt.savefig('training_curves.png')
    plt.close()

In [8]:
# 预测结果对比可视化
def plot_predictions(strs, y_true, y_pred, sample_count=50):

    mse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    plt.figure(figsize=(12, 6))
    
    plt.scatter(y_true[:sample_count], y_pred[:sample_count],
                color='blue', label=str(strs) + ' (n='+str(len(y_true))+')', alpha=0.6)
    
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], linewidth=2.5 , c="orange")
    
    plt.text(min(y_true)-0.2, max(max(y_true),max(y_pred))-2, '$R^{2}$ on Traing set='+str(round(r2,2)), fontsize=12)
    plt.text(min(y_true)-0.2, max(max(y_true),max(y_pred))-6, '$RMSE$ on Traing set='+str(round(mse,2)), fontsize=12)

    plt.xlabel('True values')
    plt.ylabel('Prediction values')
    plt.title('True VS Predicted Values')
    plt.legend(loc="lower right", fontsize=12)
    plt.grid(True)
    
    plt.savefig(str(strs) + ' predictions.png')
    plt.close()

In [9]:
# 主程序
if __name__ == "__main__":
    # 生成并划分数据
    X, y = generate_synthetic_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 数据标准化
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # 创建数据加载器
    train_dataset = FeatureDataset(X_train, y_train)
    test_dataset = FeatureDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)
    
    # 初始化模型
    model = TransformerRegressor()
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
    
    # 训练模型
    train_losses = train(model, train_loader, criterion, optimizer, scheduler, epochs=150)
    
    # 可视化训练过程
    plot_loss_curve(train_losses)

    
    # 测试模型
    model.eval()
    with torch.no_grad():
        test_preds = model(torch.FloatTensor(X_test))
        test_loss = criterion(test_preds, torch.FloatTensor(y_test))
        print(f'Test MSE: {test_loss.item():.4f}')
        plot_predictions('Test set', y_test, test_preds)
        
        train_preds = model(torch.FloatTensor(X_train))
        train_loss = criterion(train_preds, torch.FloatTensor(y_train))
        print(f'Train MSE: {train_loss.item():.4f}')
        plot_predictions('Train set', y_train, train_preds)

Epoch 1 | Loss: 35.10
Epoch 2 | Loss: 65.78
Epoch 3 | Loss: 109.49
Epoch 4 | Loss: 58.54
Epoch 5 | Loss: 51.52
Epoch 6 | Loss: 33.90
Epoch 7 | Loss: 17.53
Epoch 8 | Loss: 41.85
Epoch 9 | Loss: 16.95
Epoch 10 | Loss: 25.85
Epoch 11 | Loss: 11.09
Epoch 12 | Loss: 21.08
Epoch 13 | Loss: 13.38
Epoch 14 | Loss: 24.30
Epoch 15 | Loss: 20.92
Epoch 16 | Loss: 5.34
Epoch 17 | Loss: 4.86
Epoch 18 | Loss: 23.46
Epoch 19 | Loss: 10.36
Epoch 20 | Loss: 36.50
Epoch 21 | Loss: 18.45
Epoch 22 | Loss: 7.38
Epoch 23 | Loss: 11.24
Epoch 24 | Loss: 8.40
Epoch 25 | Loss: 3.98
Epoch 26 | Loss: 11.57
Epoch 27 | Loss: 6.95
Epoch 28 | Loss: 9.81
Epoch 29 | Loss: 5.84
Epoch 30 | Loss: 9.34
Epoch 31 | Loss: 10.17
Epoch 32 | Loss: 9.39
Epoch 33 | Loss: 6.83
Epoch 34 | Loss: 7.82
Epoch 35 | Loss: 3.14
Epoch 36 | Loss: 6.17
Epoch 37 | Loss: 7.33
Epoch 38 | Loss: 8.19
Epoch 39 | Loss: 5.34
Epoch 40 | Loss: 5.14
Epoch 41 | Loss: 5.14
Epoch 42 | Loss: 3.95
Epoch 43 | Loss: 2.54
Epoch 44 | Loss: 7.71
Epoch 45 | Loss: 3

####  排列重要性分析（最推荐）
通过随机打乱特征值观察模型性能变化，适用于任何模型类型

In [ ]:
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

def calculate_permutation_importance(model, X_test, y_test):
    model.eval()
    # 转换为numpy数组
    X_test_np = X_test.numpy()
    y_test_np = y_test.numpy()
    
    # 计算基准得分
    with torch.no_grad():
        baseline_score = -criterion(model(X_test), y_test).item()

    # 计算排列重要性
    result = permutation_importance(
        estimator=model,
        X=X_test_np,
        y=y_test_np,
        n_repeats=10,
        scoring=lambda model, X, y: -criterion(
            model(torch.FloatTensor(X)), 
            torch.FloatTensor(y)
        ).item(),
        random_state=42
    )
    
    return result.importances_mean, baseline_score

# 主程序添加
if __name__ == "__main__":
    # ...（训练代码后添加）
    # 计算特征重要性
    importances, baseline = calculate_permutation_importance(model, test_dataset.X, test_dataset.y)
    
    # 可视化
    plt.figure(figsize=(12, 8))
    indices = np.argsort(importances)[::-1]
    plt.title("Feature Importance via Permutation")
    plt.bar(range(X_train.shape‌:ml-citation{ref="1" data="citationList"}), importances[indices], align="center")
    plt.xticks(range(X_train.shape‌:ml-citation{ref="1" data="citationList"}), indices)
    plt.xlim([-1, X_train.shape‌:ml-citation{ref="1" data="citationList"}])
    plt.ylabel("Mean Accuracy Decrease")
    plt.show()


#### 注意力权重分析（Transformer特有）
提取Transformer注意力层的权重分布：

In [ ]:
class InterpretableTransformer(TransformerRegressor):
    def forward(self, x):
        x = x.unsqueeze(-1)
        x = self.feature_embedding(x)
        x = self.pos_encoder(x)
        attentions = []
        for layer in self.transformer.layers:
            x, attn = layer.self_attn(x, x, x, need_weights=True)
            attentions.append(attn.detach().cpu())
            x = layer(x)
        x = x.mean(dim=1)
        return self.regressor(x).squeeze(-1), attentions

# 分析函数
def visualize_attention(attentions):
    plt.figure(figsize=(15, 10))
    for i, attn in enumerate(attentions):
        plt.subplot(3, 2, i+1)
        plt.imshow(attn.mean(0), cmap='viridis')
        plt.title(f"Layer {i+1} Attention")
        plt.colorbar()
    plt.tight_layout()
    plt.show()


#### SHAP值分析（个体特征解释）

In [ ]:
import shap

def shap_analysis(model, sample_data):
    # 创建解释器
    explainer = shap.DeepExplainer(
        model,
        train_loader.dataset.X[:100]  # 背景数据集
    )
    
    # 计算SHAP值
    shap_values = explainer.shap_values(sample_data)
    
    # 可视化
    shap.summary_plot(shap_values, sample_data, feature_names=[f"F{i}" for i in range(26)])
    shap.plots.bar(shap_values, max_display=15)


#### 梯度重要性分析

In [ ]:
def gradient_feature_importance(model, sample_X):
    model.eval()
    sample_X.requires_grad = True
    
    output = model(sample_X.unsqueeze(0))
    output.backward()
    
    gradients = torch.abs(sample_X.grad)
    normalized = gradients / gradients.sum()
    
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(normalized)), normalized.detach().numpy())
    plt.title("Gradient-based Feature Importance")
    plt.xlabel("Feature Index")
    plt.ylabel("Normalized Gradient Magnitude")
    plt.show()


In [ ]:
if __name__ == "__main__":
    # ...（原有训练代码）
    
    # === 特征重要性分析 ===
    # 方法1: 排列重要性
    importances, _ = calculate_permutation_importance(model, test_dataset.X, test_dataset.y)
    
    # 方法2: 注意力可视化
    interpret_model = InterpretableTransformer()
    _, attentions = interpret_model(test_dataset.X[:1])
    visualize_attention(attentions)
    
    # 方法3: SHAP分析
    shap_analysis(model, test_dataset.X[:10])
    
    # 方法4: 梯度重要性
    sample_idx = 0
    gradient_feature_importance(model, test_dataset.X[sample_idx])
